In [ ]:
import pandas as pd
from tqdm import tqdm
from gensim.models import Word2Vec
from datetime import datetime
import numpy as np

In [ ]:
x_train = pd.read_csv('../dataset/data/x_train.csv')
x_test = pd.read_csv('../dataset/data/x_test.csv')
y_train = pd.read_csv('../dataset/data/y_train.csv')
y_test = pd.read_csv('../dataset/data/y_test.csv')

In [ ]:
x_train['item_id'] = x_train['item_id'].astype(str)
y_train['item_id'] = y_train['item_id'].astype(str)
x_test['item_id'] = x_test['item_id'].astype(str)
y_test['item_id'] = y_test['item_id'].astype(str)

In [ ]:
session_items_map = x_train.groupby('session_id')['item_id'].apply(list).to_dict()
session_purchase_map = y_train.groupby('session_id')['item_id'].apply(list).to_dict()

In [ ]:
training_set= []
for session,items in tqdm(session_items_map.items()):
  training_set.append(items+session_purchase_map[session])

100%|██████████| 918382/918382 [00:02<00:00, 317806.65it/s]


In [ ]:
max_items = len(max(training_set,key=len))
model = Word2Vec(sentences=training_set)

In [ ]:
test_session_items_map = x_test.groupby('session_id')['item_id'].apply(list).to_dict()
test_session_purchase_map = y_test.groupby('session_id')['item_id'].apply(list).to_dict()

In [ ]:
test_set= []
y_true = []
for session,items in tqdm(test_session_items_map.items()):
  test_set.append(items)
  y_true.append(test_session_purchase_map[session][0])

100%|██████████| 81618/81618 [00:00<00:00, 817765.89it/s]


In [ ]:
ranks = []
for i,sample in tqdm(enumerate(test_set)):
  preds = model.predict_output_word(sample,topn=100)
  if preds is not None:
    sorted_preds = sorted(preds,key=lambda x:x[1],reverse=True)
    vals = [val[0] for val in sorted_preds]
    if y_true[i] in vals:
      rank = vals.index(y_true[i])+1
      ranks.append(1/rank)
    else:
      ranks.append(0)
  else:
    ranks.append(0)



0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gensim/models/word2vec.py:1140: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")
81618it [11:27, 118.67it/s]


In [ ]:
mean_reciprocal_rank = sum(ranks) / len(ranks)


In [ ]:
mean_reciprocal_rank

0.06310078296938161